```
Copyright 2018 FAU-iPAT (http://ipat.uni-erlangen.de/)
```

# Model generation
This notebook generates the keras model "Tucana v5.4" which is used to find to location of peaks in a given FFT. The model takes as inputs a 6 feature thick data for real, imaginary and absolute value of 2 different window functions. The output is a 256 binary vector whether or not a peak was detected at the according location.

In [1]:
from keras.models import Model
from keras.layers import Input, Conv1D, Dense, Concatenate, Flatten, Dropout, MaxPooling1D, Multiply, Reshape

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Define the input tensor
The input shape is $(256, 6)$, meaning the length of the input fourier transformation is $256$ and for each point $6$ features are given. Those features are the absolute, real and imaginary compontents of the transformation for each of the rectangular and the hanning window function.

In [2]:
window_count = 2
shape_input = (256, 3*window_count)
tensor_input = Input(shape=shape_input, name='input')
tensor_data = tensor_input

### Build the initial feature preparation stem
Here two blocks of parallel width-$1$ and width-$3$ convolutions are applied. Those two blocks are intended to recombine the given features into more significant features by either recombining the features at the location or combine with the neighbouring values. Each block passes through its input features and adds to new features. Thus the number of features within those blocks is increased from $6$ features at the input to $128$ features at its output.

In [3]:
# Calculate number of new features to get to total 64 features at the end
feature_remain = 64 - int(tensor_data.shape[-1])
feature_recombine = int(feature_remain / 2)
feature_conv = feature_remain - feature_recombine
# Create convolutions layers and concatenate them with input features
tensor_recombine = Conv1D(feature_recombine, 1, activation='relu', padding='same', name='recombine1')(tensor_data)
tensor_conv = Conv1D(feature_conv, 3, activation='relu', padding='same', name='conv1')(tensor_data)
tensor_data = Concatenate(name='data2')([tensor_data, tensor_recombine, tensor_conv])

In [4]:
# Create convolutions layers and concatenate them with input features
tensor_recombine = Conv1D(32, 1, activation='relu', padding='same', name='recombine2')(tensor_data)
tensor_conv = Conv1D(32, 3, activation='relu', padding='same', name='conv2')(tensor_data)
tensor_data = Concatenate(name='data3')([tensor_data, tensor_recombine, tensor_conv])

### Build the inception layers
Here three Inception blocks are attached to the previous layers. Each of the blocks is identically created by the following function. As can be seen in the function the Inception blocks contain 4 parallel branches (Maxpooling, width-$1$ convolution, width-$3$ convolution and width-$5$ convolution).

In [5]:
# Function to add an Inception like block to the input tensor
def make_inception(prefix, tensor_input):
    # MaxPooling branch
    tensor_pool = MaxPooling1D(3, strides=1, padding='same', name=prefix+'pool1')(tensor_input)
    tensor_pool = Conv1D(32, 1, activation='relu', padding='same', name=prefix+'pool2')(tensor_pool)
    # 1x1 recombination branch
    tensor_1x1 = Conv1D(32, 1, activation='relu', padding='same', name=prefix+'1x1')(tensor_input)
    # 3x3 branch
    tensor_3x3 = Conv1D(16, 1, activation='relu', padding='same', name=prefix+'3x3_1')(tensor_input)
    tensor_3x3 = Conv1D(32, 3, activation='relu', padding='same', name=prefix+'3x3_2')(tensor_3x3)
    # 5x5 branch
    tensor_5x5 = Conv1D(16, 1, activation='relu', padding='same', name=prefix+'5x5_1')(tensor_input)
    tensor_5x5 = Conv1D(32, 5, activation='relu', padding='same', name=prefix+'5x5_2')(tensor_5x5)
    # Combine all branches
    return Concatenate(name=prefix+'result')([tensor_pool, tensor_1x1, tensor_3x3, tensor_5x5])

In [6]:
# Attach the three Inception blocks to the neural network
tensor_data = make_inception('inception1_', tensor_data)
tensor_data = make_inception('inception2_', tensor_data)
tensor_data = make_inception('inception3_', tensor_data)

### Build the final logic layers
The final layers of the neural network are used to reduce the dimensionality of the features from the $128$ features at the input of the block to the single output feature at the end. Also dropout is applied within the block to prevent overfitting of the network to the training data.

In [7]:
tensor_reduce = Conv1D(32, 1, activation='relu', padding='same', name='reduce')(tensor_data)
tensor_data = Dropout(0.5, name='dropout1')(tensor_reduce)
tensor_data = Conv1D(24, 3, activation='relu', padding='same', name='final1')(tensor_data)
tensor_data = Dropout(0.5, name='dropout2')(tensor_data)
tensor_data = Conv1D(16, 3, activation='relu', padding='same', name='final2')(tensor_data)
tensor_data = Conv1D(1, 1, activation='sigmoid', padding='same', name='final3')(tensor_data)
tensor_result = Reshape((256,), name='output')(tensor_data)

### Build the model

In [8]:
model = Model(tensor_input, tensor_result, name='TucanaV5.4')
print('Number of layers = {}'.format(len(model.layers)))
model.summary()

Number of layers = 38
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 256, 6)        0                                            
____________________________________________________________________________________________________
recombine1 (Conv1D)              (None, 256, 29)       203         input[0][0]                      
____________________________________________________________________________________________________
conv1 (Conv1D)                   (None, 256, 29)       551         input[0][0]                      
____________________________________________________________________________________________________
data2 (Concatenate)              (None, 256, 64)       0           input[0][0]                      
                                                                   re

### Save the model as JSON

In [9]:
json = model.to_json()
with open("./tucana_v5.4.json", "w") as file:
    file.write(json)

```
Copyright 2018 FAU-iPAT (http://ipat.uni-erlangen.de/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```